In [ ]:
import matplotlib.pyplot as plt
import fenics as fe
import mshr
import numpy as np
from scipy.linalg import eigh

In [ ]:
# Constants
CENTER = fe.Point(0, 0)
RADIUS = 1
DOMAIN = mshr.Circle(CENTER, RADIUS)
DIRICHLET_BC = fe.Constant(0)

In [ ]:
def cov1_1(x, y):
    return 5.0/100.0 * np.exp(-4.0 * ((x[0] - y[0])**2 + (x[1] - y[1])**2))
def cov1_2(x, y):
    return 1.0/100.0 * np.exp(-0.1 * ((2*x[0] - y[0])**2 + (2*x[1] - y[1])**2))
def cov2_1(x, y):
    return 1.0/100.0 * np.exp(-0.1 * ((x[0] - 2*y[0])**2 + (x[1] - 2*y[1])**2))
def cov2_2(x, y):
    return 5.0/100.0 * np.exp(-1.0 * ((x[0] - y[0])**2 + (x[1] - y[1])**2))

In [ ]:
# Helpers eigenpair calculation
class BasisFunction():
    def __init__(self, basis_function: fe.Function, coordinates: np.array):
        self.function = basis_function
        self.coordinates = coordinates

def get_max_edge_length(mesh):
    max_length = 0
    for cell in fe.cells(mesh):
        vertices = np.array(cell.get_vertex_coordinates()).reshape((-1, 2))
        edge_lengths = np.linalg.norm(np.roll(vertices, -1, axis=0) - vertices, axis=1)
        max_length = max(max_length, np.max(edge_lengths))
    return max_length

def get_C_entry(mc_samples_for_c_entries, f, basis_function_i: BasisFunction, basis_function_j: BasisFunction, max_edge_length: float):
    def integrand(x, y):
        return f(x, y) * basis_function_i.function(x) * basis_function_j.function(y)
    # generate MC samples
    angles_x = np.random.uniform(0, 2 * np.pi, mc_samples_for_c_entries)
    radii_x = np.sqrt(np.random.uniform(0, max_edge_length, mc_samples_for_c_entries))
    x1_samples = basis_function_i.coordinates[0] + radii_x * np.cos(angles_x)
    x2_samples = basis_function_i.coordinates[1] + radii_x * np.sin(angles_x)
    angles_y = np.random.uniform(0, 2 * np.pi, mc_samples_for_c_entries)
    radii_y = np.sqrt(np.random.uniform(0, max_edge_length, mc_samples_for_c_entries))
    y1_samples = basis_function_j.coordinates[0] + radii_y * np.cos(angles_y)
    y2_samples = basis_function_j.coordinates[1] + radii_y * np.sin(angles_y)
    C_entry = 0
    for (x1, x2) in zip(x1_samples, x2_samples):
        for (y1, y2) in zip(y1_samples, y2_samples):
            C_entry += integrand([x1, x2], [y1, y2])
    return C_entry / (mc_samples_for_c_entries**2) * np.pi * max_edge_length**2

def calculate_vector_field_eigenpairs(mesh_resolution_c_entries, mc_samples_for_c_entries):  
    mesh = mshr.generate_mesh(DOMAIN, mesh_resolution_c_entries)
    max_edge_length = get_max_edge_length(mesh)
    V = fe.FunctionSpace(mesh, "CG", 1)
    V_Vector = fe.VectorFunctionSpace(mesh, "CG", 1)
    N = V.dim()
    dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, 2))

    basis_functions = []
    basis_functions_grads = []
    for i in range(N):
        basis_function = fe.Function(V)
        basis_function.vector()[i] = 1.0
        basis_function.set_allow_extrapolation(True)
        basis_functions.append(BasisFunction(basis_function, dof_coordinates[i]))
        grad = fe.project(fe.grad(basis_function), V_Vector)
        grad.set_allow_extrapolation(True)
        basis_functions_grads.append(BasisFunction(grad, dof_coordinates[i]))

    C = np.zeros((2 * N, 2 * N))
    for i, basis_function_i in enumerate(basis_functions):
        for j, basis_function_j in enumerate(basis_functions):
            if j <= i:
                # Here we use that each block is symmetric because of the symmetry of the covariance functions
                C[i, j] = C[j, i] = get_C_entry(mc_samples_for_c_entries, cov1_1, basis_function_i, basis_function_j, max_edge_length)
                C[i, N + j] = C[j, N + i] = get_C_entry(mc_samples_for_c_entries, cov1_2, basis_function_i, basis_function_j, max_edge_length)
                C[N + i, j] = C[N + j, i] = get_C_entry(mc_samples_for_c_entries, cov2_1, basis_function_i, basis_function_j, max_edge_length)
                C[N + i, N + j] = C[N + j, N + i] = get_C_entry(mc_samples_for_c_entries, cov2_2, basis_function_i, basis_function_j, max_edge_length)
    # print(f"C: {C}")

    M = np.zeros((2 * N, 2 * N))
    for i, basis_function in enumerate(basis_functions):
        integrand = basis_function.function * basis_function.function * fe.dx
        M[i, i] = M[N + i, N + i] = fe.assemble(integrand)
    # print(f"M: {M}")


    J = N # Number of eigenvectors -> J = N is maximum
    eigenvalues, eigenvectors = eigh(C, M, subset_by_index=[0, J-1])
    # Print the eigenvalues and eigenvectors -> important to test if MC-sample size is large enough
    # print(f"Eigenvalues: {eigenvalues}")
    # print(f"Eigenvectors: {eigenvectors}")

    # Eliminate negative eigenvalues
    for index, eigenvalue in enumerate(eigenvalues):
        if eigenvalue < 0:
            eigenvalues[index] = 0
    return eigenvalues, eigenvectors, basis_functions_grads, N, J

In [ ]:
###### Section 1 ######
###### Calculation of the eigenpairs for the random field ###### 

# Inputs
# 3, 100 takes approximately 5 minutes
mesh_resolution_c_entries = 3
mc_samples_for_c_entries = 100

# Calculate the eigenpairs
eigenvalues, eigenvectors, basis_functions_grads, N, J = calculate_vector_field_eigenpairs(mesh_resolution_c_entries, mc_samples_for_c_entries)

In [ ]:
# Helpers sampling
def random_field(x, eigenvalues, eigenvectors, xi, basis_functions, N, J):
    return x[0] + sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[k, j] * basis_functions[k].function(x) for k in range(N)]) * xi[j] for j in range(J)]), \
           x[1] + sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[N + k, j] * basis_functions[k].function(x) for k in range(N)]) * xi[j] for j in range(J)])

def jacobian(x, eigenvalues, eigenvectors, xi, basis_functions_grads, N, J):
    jacobian_output = np.zeros((2, 2))
    jacobian_output[0, 0] = 1 + sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[k, j] * basis_functions_grads[k].function(x)[0] for k in range(N)]) * xi[j] for j in range(J)])
    jacobian_output[0, 1] = sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[N + k, j] * basis_functions_grads[k].function(x)[0] for k in range(N)]) * xi[j] for j in range(J)])
    jacobian_output[1, 0] = sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[k, j] * basis_functions_grads[k].function(x)[1] for k in range(N)]) * xi[j] for j in range(J)])
    jacobian_output[1, 1] = 1 + sum([np.sqrt(eigenvalues[j]) * sum([eigenvectors[N + k, j] * basis_functions_grads[k].function(x)[0] for k in range(N)]) * xi[j] for j in range(J)])
    return jacobian_output

class AExpression(fe.UserExpression):
    def __init__(self, eigenvalues, eigenvectors, xi, basis_functions_grads, N, J, **kwargs):
        super().__init__(**kwargs)
        self.eigenvalues = eigenvalues
        self.eigenvectors = eigenvectors
        self.xi = xi
        self.basis_functions_grads = basis_functions_grads
        self.N = N
        self.J = J

    def eval(self, values, x):
        J_x = jacobian(x, self.eigenvalues, self.eigenvectors, self.xi, self.basis_functions_grads, self.N, self.J)
        inv_JTJ = np.linalg.inv(J_x.T @ J_x)
        det_J = np.linalg.det(J_x)
        A_x = inv_JTJ * det_J
        values[0] = A_x[0, 0]
        values[1] = A_x[0, 1]
        values[2] = A_x[1, 0]
        values[3] = A_x[1, 1]

    def value_shape(self):
        return (2, 2)
    
def solve_poisson_for_given_sample(mesh_resolution_fem_single, eigenvalues, eigenvectors, xi, basis_functions_grads, N, J, f):
    mesh = mshr.generate_mesh(DOMAIN, mesh_resolution_fem_single)
    V = fe.FunctionSpace(mesh, "CG", 1)
    u = fe.TrialFunction(V)
    v = fe.TestFunction(V)
    A_expr = AExpression(eigenvalues, eigenvectors, xi, basis_functions_grads, N, J, degree=2)
    a = fe.inner(fe.dot(A_expr, fe.grad(u)), fe.grad(v)) * fe.dx
    L = f * v * fe.dx
    bc = fe.DirichletBC(V, DIRICHLET_BC, 'on_boundary')
    u_sol = fe.Function(V)
    fe.solve(a == L, u_sol, bc)
    return u_sol

def mc_single_global_calculation(mc_samples_single, mesh_resolution_fem_single):
    u_sols = []
    for i in range(mc_samples_single):
        print(f"Iteration {i+1}/{mc_samples_single}")
        xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), J)
        u_sols.append(solve_poisson_for_given_sample(mesh_resolution_fem_single, eigenvalues, eigenvectors, xi, basis_functions_grads, N, J))

    mesh = mshr.generate_mesh(DOMAIN, mesh_resolution_fem_single)
    V = fe.FunctionSpace(mesh, "CG", 1)
    mean_sol = fe.Function(V)
    mean_sol.vector()[:] = 0
    for u_sol in u_sols:
        mean_sol.vector()[:] += u_sol.vector() / mc_samples_single
    return mean_sol

def calc_sobol_index_123(mc_sample_size, mesh_resolution_fem, size_of_xi, P):

    # Implementation of the double loop MC estimation of specific Sobol Indix
    for i in range(mc_sample_size):
        mean_sols_point_P = np.zeros(len(mc_sample_size))
        sols_point_P = []
        xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), size_of_xi) # in algorithm x_A
        for k in range(mc_sample_size):
            f = np.random.normal(0, 1) # in algorithm x_A_comp
            sols_point_P.append(solve_poisson_for_given_sample(mesh_resolution_fem, eigenvalues, eigenvectors, xi, basis_functions_grads, size_of_xi, J, f))
        mean_sols_point_P[i] = np.mean(sols_point_P)
    y_bar = np.mean(mean_sols_point_P)
    #TODO line 11 in algorithm ...
    #TODO check if eigenvalues are ordered correctly! 



In [ ]:
###### Section 2 ######
###### Single MC-loop Calculation of samples of the vector field saved globally ######

# Inputs
mc_sample_size = 10
mesh_resolution_fem = 4
size_of_xi = 3
P = fe.Point(0, 0)

# Calling the function
sobol_index_123 = calc_sobol_index_123(mc_sample_size, mesh_resolution_fem, size_of_xi, P)
print(f"Sobol indix 123: {sobol_index_123}")